In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

: 

In [ ]:
# --- ۱. تعریف پارامترهای نمادین (Symbolic Parameters) ---
theta1, theta2, theta3, theta4, theta5 = sp.symbols('theta1 theta2 theta3 theta4 theta5')
d1, d2, d3, d4, d5 = sp.symbols('d1 d2 d3 d4 d5')
a1, a2, a3, a4, a5 = sp.symbols('a1 a2 a3 a4 a5')
alpha1, alpha2, alpha3, alpha4, alpha5 = sp.symbols('alpha1 alpha2 alpha3 alpha4 alpha5')

In [ ]:
def get_dh_matrix(theta, d, a, alpha):
    """محاسبه ماتریس تبدیل همگن استاندارد DH برای یک لینک"""
    return sp.Matrix([
        [sp.cos(theta), -sp.sin(theta)*sp.cos(alpha),  sp.sin(theta)*sp.sin(alpha), a*sp.cos(theta)],
        [sp.sin(theta),  sp.cos(theta)*sp.cos(alpha), -sp.cos(theta)*sp.sin(alpha), a*sp.sin(theta)],
        [0,              sp.sin(alpha),               sp.cos(alpha),              d],
        [0,              0,                           0,                          1]
    ])


In [ ]:
# --- ۲. مقادیر عددی لینک‌های ربات RoArm-M3 Pro (بر اساس دیتاشیت) ---
# L1 = 179.52mm (Base height), L2 = 236.82mm (Arm), L3 = 316.15mm (Forearm + Wrist)
# Offset = 30.00mm
L1, L2, L3, Offset = 179.52, 236.82, 316.15, 30.00

In [ ]:
# جدول پارامترهای DH استاندارد برای 5 درجه آزادی
# [theta, d, a, alpha]
dh_table = [
    (theta1, L1,     0,      sp.pi/2),   # مفصل 1: چرخش پایه
    (theta2, Offset, L2,     0),         # مفصل 2: شانه
    (theta3, 0,      L3,     0),         # مفصل 3: آرنج
    (theta4, 0,      0,      -sp.pi/2),  # مفصل 4: مچ (Pitch)
    (theta5, 0,      0,      0)          # مفصل 5: مچ (Roll)
]

In [ ]:
# --- ۳. محاسبه ماتریس تبدیل کل (T0_5) ---
T0_5 = sp.eye(4)
A_matrices = []

print("در حال محاسبه ماتریس های تبدیل هر لینک...")
for i, (q, d_val, a_val, alpha_val) in enumerate(dh_table):
    Ai = get_dh_matrix(q, d_val, a_val, alpha_val)
    A_matrices.append(Ai)
    T0_5 = T0_5 * Ai

# نمایش مختصات نهایی (End-Effector) به صورت نمادین
x_ee = T0_5[0, 3]
y_ee = T0_5[1, 3]
z_ee = T0_5[2, 3]

print("\nفرمول مختصات دکارتی End-Effector:")
print(f"X = {sp.simplify(x_ee)}")
print(f"Y = {sp.simplify(y_ee)}")
print(f"Z = {sp.simplify(z_ee)}")

In [ ]:
# --- ۴. تابع محاسبات عددی (Numerical FK) ---
def calculate_fk_numerical(joint_angles_deg):
    """محاسبه موقعیت نهایی بر اساس زوایای ورودی (درجه)"""
    joint_angles_rad = np.radians(joint_angles_deg)
    # جایگذاری زوایا در ماتریس کل
    subs = {theta1: joint_angles_rad[0], theta2: joint_angles_rad[1], 
            theta3: joint_angles_rad[2], theta4: joint_angles_rad[3], 
            theta5: joint_angles_rad[4]}
    T_final = np.array(T0_5.evalf(subs=subs)).astype(np.float64)
    return T_final

# تست با زوایای صفر (حالت کشیده)
test_joints = [0, 0, 0, 0, 0]
T_test = calculate_fk_numerical(test_joints)
print(f"\nموقعیت نهایی در حالت Home (زوایای صفر): \n{T_test[:3, 3]}")

In [ ]:
# --- ۵. شبیه‌سازی فضای کاری (Workspace Simulation - Monte Carlo) ---
def plot_roarm_workspace(n_samples=1000):
    print(f"\nدر حال تولید {n_samples} نقطه برای فضای کاری...")
    x_pts, y_pts, z_pts = [], [], []
    
    # محدودیت حرکتی مفاصل بر اساس دیتاشیت
    limits = [(-180, 180), (0, 180), (-112, 112), (-67, 67), (-135, 135)]
    
    for _ in range(n_samples):
        rand_joints = [np.random.uniform(low, high) for low, high in limits]
        T = calculate_fk_numerical(rand_joints)
        x_pts.append(T[0, 3])
        y_pts.append(T[1, 3])
        z_pts.append(T[2, 3])
        
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x_pts, y_pts, z_pts, s=2, c=z_pts, cmap='viridis', alpha=0.5)
    ax.set_title("RoArm-M3 Pro Estimated Workspace")
    ax.set_xlabel("X (mm)")
    ax.set_ylabel("Y (mm)")
    ax.set_zlabel("Z (mm)")
    plt.show()

# اجرای رسم فضای کاری (اختیاری)
# plot_roarm_workspace(1500)